# Analysis of urea sensitivity of isolated cgre proteins

Importing necessary libraries:

In [ ]:
from pathlib import Path
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from loguru import logger
from scipy.optimize import curve_fit, OptimizeWarning

# Curve fitting metrics
from scipy.stats import kendalltau, linregress
from sklearn.metrics import mean_squared_error

from tqdm import tqdm

from utils import *

%matplotlib inline

Disable `RunTimeWarning` and `scipy.optimize.OptimizeWarning`:

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=OptimizeWarning)

Conversion of genes to wells:

In [ ]:
blanks_urea = [f"{col}1" for col in "ABCDEFGH"]
blanks_pbs = [f"{col}12" for col in "ABCDEFGH"]
wells_urea = [f"{col}{i}" for col in "ABCDEFGH" for i in range(2, 7)]
wells_pbs = [f"{col}{i}" for col in "ABCDEFGH" for i in range(7, 12)]

gene2wells_urea = {
    # Plate 0
    "9708": [f"{s}3" for s in "ABCDEFGH"],
    "4111": [f"{s}4" for s in "ABCDEFGH"],
    "132": [f"{s}2" for s in "ABCDEFGH"],
    "1338": [f"{s}5" for s in "ABCDEFGH"],
    # Plate 1
    "83": ["A2", "A6", "B3", "C4", "D3", "E2", "E6", "F5", "G4", "H3"],
    "2880": ["A3", "B2", "B6", "C5", "D4", "E3", "F2", "F6", "G5", "H4"],
    "3224": ["A4", "B5", "C2", "C6", "D5", "E4", "F3", "G2", "G6", "H5"],
    "121": ["A5", "B4", "C3", "D2", "D6", "E5", "F4", "G3", "H2", "H6"],
    # Plate 2
    "626": ["A2", "A6", "B3", "C4", "D3", "E2", "E6", "F5", "G4", "H3"],
    "1414": ["A3", "B2", "B6", "C5", "D4", "E3", "F2", "F6", "G5", "H4"],
    "911": ["A4", "B5", "C2", "C6", "D5", "E4", "F3", "G2", "G6", "H5"],
    "900": ["A5", "B4", "C3", "D2", "D6", "E5", "F4", "G3", "H2", "H6"],
    # Plate 3
    "575": ["A4", "B5", "C2", "C6", "D5", "E4", "F3", "G2", "G6", "H5"],
    "985": ["A5", "B4", "C3", "D2", "D6", "E5", "F4", "G3", "H2", "H6"],
    # Plate 4
    "13": ["A3", "B2", "B6", "C5", "D4", "E3", "F2", "F6", "G5", "H4"],
    "567": ["A2", "A6", "B3", "C4", "D3", "E2", "E6", "F5", "G4", "H3"],
}

gene2wells_pbs = {
    # Plate 0
    "9708": [f"{s}10" for s in "ABCDEFGH"],
    "4111": [f"{s}9" for s in "ABCDEFGH"],
    "132": [f"{s}11" for s in "ABCDEFGH"],
    "1338": [f"{s}8" for s in "ABCDEFGH"],
    # Plate 1
    "83": ["A7", "A11", "B10", "C9", "D8", "E7", "E11", "F10", "G9", "H8"],
    "2880": ["A8", "B7", "B11", "C10", "D9", "E8", "F7", "F11", "G10", "H9"],
    "3224": ["A9", "B8", "C7", "C11", "D10", "E9", "F8", "G7", "G11", "H10"],
    "121": ["A10", "B9", "C8", "D7", "D11", "E10", "F9", "G8", "H7", "H11"],
    # Plate 2
    "626": ["A7", "A11", "B10", "C9", "D8", "E7", "E11", "F10", "G9", "H8"],
    "1414": ["A8", "B7", "B11", "C10", "D9", "E8", "F7", "F11", "G10", "H9"],
    "911": ["A9", "B8", "C7", "C11", "D10", "E9", "F8", "G7", "G11", "H10"],
    "900": ["A10", "B9", "C8", "D7", "D11", "E10", "F9", "G8", "H7", "H11"],
    # Plate 3
    "575": ["A9", "B8", "C7", "C11", "D10", "E9", "F8", "G7", "G11", "H10"],
    "985": ["A10", "B9", "C8", "D7", "D11", "E10", "F9", "G8", "H7", "H11"],
    # Plate 4
    "13": ["A7", "A11", "B10", "C9", "D8", "E7", "E11", "F10", "G9", "H8"],
    "567": ["A8", "B7", "B11", "C10", "D9", "E8", "F7", "F11", "G10", "H9"],
}

# Plate 3 - Fluorescence, and Plate 4 - only 567
gene2wells_urea_fl = {
    "13": ["A2", "A6", "B3", "C2", "D3", "E2", "E6", "F5", "G4", "H3"],
    "567": ["A2", "A6", "B3", "C4", "D3", "E2", "E6", "F5", "G4", "H3"],
    "575": ["A4", "B5", "C4", "C6", "D5", "E4", "F3", "G2", "G6", "H5"],
    "985": ["A5", "B4", "C3", "D2", "D6", "E5", "F4", "G3", "H2", "H6"],
}

gene2wells_pbs_fl4 = {
    "567": ["A7", "A11", "B10", "C9", "D8", "E7", "E11", "F10", "G9", "H8"],
}

gene2wells_fl = {}
for gene, gene_wells_pbs in gene2wells_pbs.items():
    if gene not in ["13", "567", "575", "985"]:
        gene2wells_fl[gene] = gene2wells_urea[gene] + gene_wells_pbs
    else:
        if gene == "567":
            gene2wells_fl[gene] = gene2wells_urea_fl[gene] + gene2wells_pbs_fl4[gene]
        else:
            gene2wells_fl[gene] = gene2wells_urea_fl[gene] + gene_wells_pbs

gene2wells_abs = {
    gene: gene2wells_urea[gene] + gene_wells_pbs
    for gene, gene_wells_pbs in gene2wells_pbs.items()
}

## 1. Processing raw data and initial plotting

Functions for reading excel files, substracting blanks and plotting initial spectra can be found in `utils.py`.

## 2. Plotting Absorbance

Reading Absorbance from all 3 plates:

In [ ]:
abs0 = process_raw_excel(
    [
        "data/210518_abs_urea_200ul_newclytia_1.xlsx",
        "data/210518_abs_urea_200ul_newclytia_2.xlsx",
        "data/210518_abs_urea_200ul_newclytia_3.xlsx",
    ],
    n_rows_skip=792,
)
abs1 = process_raw_excel(
    [
        "data/220531_urea_200ul_abs__cgre83-2880-3224-121___data1.xlsx",
        "data/220531_urea_200ul_abs__cgre83-2880-3224-121___data2.xlsx",
        "data/220531_urea_200ul_abs__cgre83-2880-3224-121___data3.xlsx",
        "data/220531_urea_200ul_abs__cgre83-2880-3224-121___data4.xlsx",
    ],
    n_rows_skip=792,
)
abs2 = process_raw_excel(
    [
        "data/220610_urea_200ul_abs__cgre626-900-911-1414__data1.xlsx",
        "data/220610_urea_200ul_abs__cgre626-900-911-1414__data2.xlsx",
        "data/220610_urea_200ul_abs__cgre626-900-911-1414__data3.xlsx",
    ],
    n_rows_skip=790,
)
abs3 = process_raw_excel(
    [
        "data/220613_urea_200ul_abs__cgre13-567-575-985__data1.xlsx",
        "data/220613_urea_200ul_abs__cgre13-567-575-985__data2.xlsx",
        "data/220613_urea_200ul_abs__cgre13-567-575-985__data3.xlsx",
    ],
    n_rows_skip=790,
)
abs4 = process_raw_excel(
    [
        "data/220622_urea_200ul_abs__cgre13-567__data1.xlsx",
        "data/220622_urea_200ul_abs__cgre13-567__data2.xlsx",
        "data/220622_urea_200ul_abs__cgre13-567__data3.xlsx",
    ],
    n_rows_skip=790,
)

In [ ]:
abs0_sub = subtract_blanks(abs0, blanks_urea, blanks_pbs, wells_urea, wells_pbs)
abs1_sub = subtract_blanks(abs1, blanks_urea, blanks_pbs, wells_urea, wells_pbs)
abs2_sub = subtract_blanks(abs2, blanks_urea, blanks_pbs, wells_urea, wells_pbs)
abs3_sub = subtract_blanks(abs3, blanks_urea, blanks_pbs, wells_urea, wells_pbs)
abs4_sub = subtract_blanks(abs4, blanks_urea, blanks_pbs, wells_urea, wells_pbs)

### Plate 0:

PBS:

In [ ]:
plot_spectra(
    abs0_sub, ["9708", "4111", "132", "1338"], gene2wells_pbs, "Absorbance", urea=False
)

Urea:

In [ ]:
plot_spectra(
    abs0_sub, ["9708", "4111", "132", "1338"], gene2wells_urea, "Absorbance", urea=True
)

**WELLS TO EXCLUDE**:

PBS: A10, B10, C10, A9, B9, C9, A11, B11, A8, B8

Urea: A3, H3, A4, H4, A2, H2

### Plate 1:

PBS:

In [ ]:
plot_spectra(
    abs1_sub, ["83", "2880", "3224", "121"], gene2wells_pbs, "Absorbance", urea=False
)

Urea:

In [ ]:
plot_spectra(
    abs1_sub, ["83", "2880", "3224", "121"], gene2wells_urea, "Absorbance", urea=True
)

**WELLS TO EXCLUDE**:

PBS: B10, A8, B11, B8, B9

Urea: A6, B6, C6, D6, E6, F6, G6, H6, H4, B5, H2

### Plate 2:

PBS:

In [ ]:
plot_spectra(
    abs2_sub, ["626", "1414", "911", "900"], gene2wells_pbs, "Absorbance", urea=False
)

Urea:

In [ ]:
plot_spectra(
    abs2_sub, ["626", "1414", "911", "900"], gene2wells_urea, "Absorbance", urea=True
)

**WELLS TO EXCLUDE**:

PBS: A7, B10, C9, A8, C10, A9, B8, C11, B9, C8, H7

Urea: F6, D4, B6, F3, H6, D6

### Plate 3:

PBS:

In [ ]:
plot_spectra(
    abs3_sub, ["575", "985"], gene2wells_pbs, "Absorbance", urea=False, figsize=(40, 8)
)

Urea:

In [ ]:
plot_spectra(
    abs3_sub, ["575", "985"], gene2wells_urea, "Absorbance", urea=True, figsize=(40, 8)
)

**WELLS TO EXCLUDE**:

PBS: A9, B8, B9, A10

Urea: C6, G6, H6, D6

### Plate 4:

PBS:

In [ ]:
plot_spectra(
    abs4_sub, ["13", "567"], gene2wells_pbs, "Absorbance", urea=False, figsize=(40, 8)
)

Urea:

In [ ]:
plot_spectra(
    abs4_sub, ["13", "567"], gene2wells_urea, "Absorbance", urea=True, figsize=(40, 8)
)

**WELLS TO EXCLUDE**:

PBS: A7, A11, B10, B11, B7, A8

Urea: B6, F6

### Wild-Type (WT)

These files exist already in preprocessed format, so we can plot them directly. 

In [ ]:
absWT = pd.read_csv("data/cgreWT_absorbance.csv", index_col="Unnamed: 0").reset_index(
    drop=True
)

In [ ]:
plt.figure(figsize=(40, 8), dpi=200)
u = list(set(absWT[absWT.treatment == "urea"].replicate))
p = list(set(absWT[absWT.treatment == "pbs"].replicate))

for i in range(8):
    plt.subplot(2, 8, i + 1)
    sns.lineplot(
        data=absWT[absWT.replicate == p[i]],
        x="wavelength",
        y="signal",
        hue="time",
        palette=sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True),
    )
    plt.title(f"cgreWT ({u[i]})", fontsize=18)
    plt.ylabel("Absorbance (PBS)", fontsize=16)
    plt.xlabel("Wavelength", fontsize=16)

    plt.subplot(2, 8, i + 9)
    sns.lineplot(
        data=absWT[absWT.replicate == u[i]],
        x="wavelength",
        y="signal",
        hue="time",
        palette=sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True),
    )
    plt.title(f"cgreWT ({p[i]})", fontsize=18)
    plt.ylabel("Absorbance (9M Urea)", fontsize=16)
    plt.xlabel("Wavelength", fontsize=16)

plt.tight_layout()

## 3. Plotting Fluorescence

Reading Absorbance from all 3 plates:

In [ ]:
fl0 = process_raw_excel(
    [
        "data/210608_fluo_urea_200ul_newclytia_1.xlsx",
        "data/210608_fluo_urea_200ul_newclytia_2.xlsx",
        "data/210608_fluo_urea_200ul_newclytia_3.xlsx",
    ],
    n_rows_skip=1210,
)
fl1 = process_raw_excel(
    [
        "data/220531_fluo_200ul__cgre83-2880-3224-121__data1.xlsx",
        "data/220531_fluo_200ul__cgre83-2880-3224-121__data2.xlsx",
        "data/220531_fluo_200ul__cgre83-2880-3224-121__data3.xlsx",
    ],
    n_rows_skip=1210,
)
fl2 = process_raw_excel(
    [
        "data/220608_fluo_200ul__cgre626-900-911-1414__data1.xlsx",
        "data/220608_fluo_200ul__cgre626-900-911-1414__data2.xlsx",
        "data/220608_fluo_200ul__cgre626-900-911-1414__data3.xlsx",
    ],
    n_rows_skip=1210,
)
fl3 = process_raw_excel(
    [
        "data/220613_fluo_200ul_cgre13-567-575-985__data1.xlsx",
        "data/220613_fluo_200ul_cgre13-567-575-985__data2.xlsx",
        "data/220613_fluo_200ul_cgre13-567-575-985__data3.xlsx",
    ],
    n_rows_skip=1210,
)
fl4 = process_raw_excel(
    [
        "data/220622_fluo_200ul_cgre567_and_10Mcomps__data1.xlsx",
        "data/220622_fluo_200ul_cgre567_and_10Mcomps__data2.xlsx",
        "data/220622_fluo_200ul_cgre567_and_10Mcomps__data3.xlsx",
    ],
    n_rows_skip=1210,
)

In [ ]:
fl0_sub = subtract_blanks(fl0, blanks_urea, blanks_pbs, wells_urea, wells_pbs)
fl1_sub = subtract_blanks(fl1, blanks_urea, blanks_pbs, wells_urea, wells_pbs)
fl2_sub = subtract_blanks(fl2, blanks_urea, blanks_pbs, wells_urea, wells_pbs)
fl3_sub = subtract_blanks(fl3, blanks_urea, blanks_pbs, wells_urea, wells_pbs)
fl4_sub = subtract_blanks(fl4, blanks_urea, blanks_pbs, wells_urea, wells_pbs)

### Plate 0:

PBS:

In [ ]:
plot_spectra(
    fl0_sub, ["9708", "4111", "132", "1338"], gene2wells_pbs, "Fluorescence", urea=False
)

Urea:

In [ ]:
plot_spectra(
    fl0_sub, ["9708", "4111", "132", "1338"], gene2wells_urea, "Fluorescence", urea=True
)

**WELLS TO EXCLUDE**:

PBS: A10, H10, A9, H9, A8, H8, A11

Urea:

Louisa says None, I agree, was too harsh. 

### Plate 1:

PBS:

In [ ]:
plot_spectra(
    fl1_sub, ["83", "2880", "3224", "121"], gene2wells_pbs, "Fluorescence", urea=False
)

Urea:

In [ ]:
plot_spectra(
    fl1_sub, ["83", "2880", "3224", "121"], gene2wells_urea, "Fluorescence", urea=True
)

**WELLS TO EXCLUDE**:

PBS: 

Urea: B6, F6, D6, H6


### Plate 2:

PBS:

In [ ]:
plot_spectra(
    fl2_sub, ["626", "1414", "911", "900"], gene2wells_pbs, "Fluorescence", urea=False
)

Urea:

In [ ]:
plot_spectra(
    fl2_sub, ["626", "1414", "911", "900"], gene2wells_urea, "Fluorescence", urea=True
)

**WELLS TO EXCLUDE**:

PBS:

Urea: 


### Plate 3:

PBS:

In [ ]:
plot_spectra(
    fl3_sub,
    ["13", "575", "985"],
    gene2wells_pbs,
    "Fluorescence",
    urea=False,
    figsize=(40, 12),
)

Urea:

In [ ]:
gene2wells_urea_fl3 = {
    "13": ["A2", "A6", "B3", "C2", "D3", "E2", "E6", "F5", "G4", "H3"],
    "567": ["A3", "B2", "B6", "C5", "D4", "E3", "F2", "F6", "G5", "H4"],
    "575": ["A4", "B5", "C4", "C6", "D5", "E4", "F3", "G2", "G6", "H5"],
    "985": ["A5", "B4", "C3", "D2", "D6", "E5", "F4", "G3", "H2", "H6"],
}
plot_spectra(
    fl3_sub,
    ["13", "575", "985"],
    gene2wells_urea_fl3,
    "Fluorescence",
    urea=True,
    figsize=(40, 12),
)

**WELLS TO EXCLUDE**:

PBS:

Urea: A6, E6, D6, H6

### Plate 4:

PBS:

In [ ]:
gene2wells_urea_fl = {
    "13": ["A2", "A6", "B3", "C2", "D3", "E2", "E6", "F5", "G4", "H3"],
    "567": ["A2", "A6", "B3", "C4", "D3", "E2", "E6", "F5", "G4", "H3"],
    "575": ["A4", "B5", "C4", "C6", "D5", "E4", "F3", "G2", "G6", "H5"],
    "985": ["A5", "B4", "C3", "D2", "D6", "E5", "F4", "G3", "H2", "H6"],
}

gene2wells_pbs_fl4 = {
    "567": ["A7", "A11", "B10", "C9", "D8", "E7", "E11", "F10", "G9", "H8"],
}
plot_spectra(
    fl4_sub, ["567"], gene2wells_pbs_fl4, "Fluorescence", urea=False, figsize=(40, 4)
)

Urea:

In [ ]:
plot_spectra(
    fl1_sub, ["567"], gene2wells_urea_fl, "Fluorescence", urea=True, figsize=(40, 4)
)

**WELLS TO EXCLUDE**:

PBS: C9

Urea: 


### Wild-Type (WT)

In [ ]:
flWT = pd.read_csv("data/cgreWT_fluorescence.csv", index_col="Unnamed: 0").reset_index(
    drop=True
)

In [ ]:
plt.figure(figsize=(40, 8), dpi=200)
u = list(set(flWT[flWT.treatment == "urea"].replicate))
p = list(set(flWT[flWT.treatment == "pbs"].replicate))

for i in range(8):
    plt.subplot(2, 8, i + 1)
    sns.lineplot(
        data=flWT[flWT.replicate == p[i]],
        x="wavelength",
        y="signal",
        hue="time",
        palette=sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True),
    )
    plt.title(f"cgreWT ({u[i]})", fontsize=18)
    plt.ylabel("Absorbance (PBS)", fontsize=16)
    plt.xlabel("Wavelength", fontsize=16)

    plt.subplot(2, 8, i + 9)
    sns.lineplot(
        data=flWT[flWT.replicate == u[i]],
        x="wavelength",
        y="signal",
        hue="time",
        palette=sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True),
    )
    plt.title(f"cgreWT ({p[i]})", fontsize=18)
    plt.ylabel("Absorbance (9M Urea)", fontsize=16)
    plt.xlabel("Wavelength", fontsize=16)

plt.tight_layout()

# Excluding the wells that are outliers

In [ ]:
exclude_abs0 = [
    "A10",
    "B10",
    "C10",
    "A9",
    "B9",
    "C9",
    "A11",
    "B11",
    "A8",
    "B8",
    "A3",
    "H3",
    "A4",
    "H4",
    "A2",
    "H2",
]
exclude_abs1 = [
    "B10",
    "A8",
    "B11",
    "B8",
    "B9",
    "A6",
    "B6",
    "C6",
    "D6",
    "E6",
    "F6",
    "G6",
    "H6",
    "H4",
    "B5",
    "H2",
]
exclude_abs2 = [
    "A7",
    "B10",
    "C9",
    "A8",
    "C10",
    "A9",
    "B8",
    "C11",
    "B9",
    "C8",
    "H7",
    "F6",
    "D4",
    "B6",
    "F3",
    "H6",
    "D6",
]
exclude_abs3 = ["A9", "B8", "B9", "A10", "C6", "G6", "H6", "D6"]
exclude_abs4 = ["A7", "A11", "B10", "B11", "B7", "A8", "B6", "F6"]


exclude_fl0 = []
exclude_fl1 = ["B6", "F6", "D6", "H6"]
exclude_fl2 = []
exclude_fl3 = ["A6", "E6", "D6", "H6"]
exclude_fl4 = ["C9"]

In [ ]:
wells2gene_0_abs = wells2genes(
    ["9708", "4111", "132", "1338"], gene2wells_abs, exclude_abs0
)
wells2gene_1_abs = wells2genes(
    ["83", "2880", "3224", "121"], gene2wells_abs, exclude_abs1
)
wells2gene_2_abs = wells2genes(
    ["626", "900", "911", "1414"], gene2wells_abs, exclude_abs2
)
wells2gene_3_abs = wells2genes(["575", "985"], gene2wells_abs, exclude_abs3)
wells2gene_4_abs = wells2genes(["13", "567"], gene2wells_abs, exclude_abs4)


abs_plt_wells2gene_data = {
    0: [wells2gene_0_abs, abs0_sub],
    1: [wells2gene_1_abs, abs1_sub],
    2: [wells2gene_2_abs, abs2_sub],
    3: [wells2gene_3_abs, abs3_sub],
    4: [wells2gene_4_abs, abs4_sub],
}

In [ ]:
wells2gene_0_fl = wells2genes(
    ["9708", "4111", "132", "1338"], gene2wells_fl, exclude_fl0
)
wells2gene_1_fl = wells2genes(["83", "2880", "3224", "121"], gene2wells_fl, exclude_fl1)
wells2gene_2_fl = wells2genes(["626", "900", "911", "1414"], gene2wells_fl, exclude_fl2)
wells2gene_3_fl = wells2genes(["13", "575", "985"], gene2wells_fl, exclude_fl3)
wells2gene_4_fl = wells2genes(["567"], gene2wells_fl, exclude_fl4)


fl_plt_wells2gene_data = {
    0: [wells2gene_0_fl, fl0_sub],
    1: [wells2gene_1_fl, fl1_sub],
    2: [wells2gene_2_fl, fl2_sub],
    3: [wells2gene_3_fl, fl3_sub],
    4: [wells2gene_4_fl, fl4_sub],
}

In [ ]:
spectra_abs = conc_spectra(abs_plt_wells2gene_data)
spectra_fl = conc_spectra(fl_plt_wells2gene_data)

In [ ]:
spectra_abs.head()

# Normalizing

First we take the first cycle for each well, and take the maximum absorbance across the measured wavelengths.

In [ ]:
w = set(spectra_abs.replicate)  # Set of all wells
max_abs_by_well = {
    well: spectra_abs[(spectra_abs.cycle == 0) & (spectra_abs.replicate == well)][
        "signal"
    ].max()
    for well in w
}

w = set(spectra_fl.replicate)  # Set of all wells
max_fl_by_well = {
    well: spectra_fl[(spectra_fl.cycle == 0) & (spectra_fl.replicate == well)][
        "signal"
    ].max()
    for well in w
}

For each gene we take the wavelength at which the absorbance was maximal:

In [ ]:
# Set of all genes, for absorbance and for fluorescence
genes_abs = set(spectra_abs.gene)
genes_fl = set(spectra_fl.gene)

peaks_abs = {
    gene: sorted(
        zip(
            spectra_abs[(spectra_abs.cycle == 0) & (spectra_abs.gene == gene)][
                "signal"
            ],
            spectra_abs[(spectra_abs.cycle == 0) & (spectra_abs.gene == gene)][
                "wavelength"
            ],
        ),
        reverse=True,
    )[0][1]
    for gene in genes_abs
}

peaks_fl = {
    gene: sorted(
        zip(
            spectra_fl[(spectra_fl.cycle == 0) & (spectra_fl.gene == gene)]["signal"],
            spectra_fl[(spectra_fl.cycle == 0) & (spectra_fl.gene == gene)][
                "wavelength"
            ],
        ),
        reverse=True,
    )[0][1]
    for gene in genes_fl
}


# WT cgre, previous data
peaks_abs["cgre"] = 485
peaks_fl["cgre"] = 500

We create two new columns:
* `peak` - True, if it's first cycle and the absorbance on this wavelength is highest
* `signal_normalized` - normalized signal, where the signal is divided by the highest signal on cycle 0

In [ ]:
spectra_abs["peak"] = spectra_abs[["gene", "wavelength"]].apply(
    lambda x: True if x[1] == peaks_abs[x[0]] else False, axis=1
)
spectra_abs["signal_normalized"] = spectra_abs[["replicate", "signal"]].apply(
    lambda x: x[1] / max_abs_by_well[x[0]], axis=1
)

spectra_fl["peak"] = spectra_fl[["gene", "wavelength"]].apply(
    lambda x: True if x[1] == peaks_fl[x[0]] else False, axis=1
)
spectra_fl["signal_normalized"] = spectra_fl[["replicate", "signal"]].apply(
    lambda x: x[1] / max_fl_by_well[x[0]], axis=1
)

## Merging WT and the rest

In [ ]:
absWT.rename({"time": "cycle"}, axis=1, inplace=True)
flWT.rename({"time": "cycle"}, axis=1, inplace=True)

In [ ]:
spectra_abs = pd.concat([spectra_abs, absWT]).reset_index(drop=True)
spectra_fl = pd.concat([spectra_fl, flWT]).reset_index(drop=True)

# Plotting the absorbance and fluorescence curves for each gene

In [ ]:
fig = plt.figure(figsize=(9, 15), dpi=200)
i = 1
for gene in [
    2880,
    575,
    83,
    121,
    13,
    567,
    3224,
    900,
    626,
    985,
    911,
    1414,
    "cgre",
    1338,
    132,
    9708,
    4111,
]:
    plot = plt.subplot(6, 3, i)

    sns.lineplot(
        data=spectra_abs[
            (spectra_abs.gene == str(gene)) & (spectra_abs.treatment == "pbs")
        ],
        x="wavelength",
        y="signal_normalized",
        hue="cycle",
        legend=False,
        linewidth=2,
        ci=None,
        alpha=0.5,
        palette="mako_r",
    )

    # Share the x axis
    plot_fl = plt.twinx()
    sns.lineplot(
        data=spectra_fl[
            (spectra_fl.gene == str(gene)) & (spectra_fl.treatment == "pbs")
        ],
        x="wavelength",
        y="signal_normalized",
        hue="cycle",
        legend=False,
        linewidth=2,
        ci=None,
        alpha=0.5,
        palette=sns.cubehelix_palette(start=0, rot=-0.5, as_cmap=True),
    )

    plt.title("cgreWT" if gene == "cgre" else f"cgre{gene}", fontsize=18)
    plot.set_xlabel("Wavelength [nm]", fontsize=14)
    plot.set_ylabel("Signal (absorbance)", fontsize=14)
    plot_fl.set_ylabel("Signal (fluorescence)", fontsize=14)
    i += 1

plt.tight_layout()
plt.savefig("fl_abs_timeline_pbs.png", dpi=300)

In [ ]:
plt.figure(figsize=(9, 15), dpi=200)
i = 1
for gene in [
    2880,
    575,
    83,
    121,
    13,
    567,
    3224,
    900,
    626,
    985,
    911,
    1414,
    "cgre",
    1338,
    132,
    9708,
    4111,
]:
    plot = plt.subplot(6, 3, i)

    sns.lineplot(
        data=spectra_abs[
            (spectra_abs.gene == str(gene)) & (spectra_abs.treatment == "urea")
        ],
        x="wavelength",
        y="signal_normalized",
        hue="cycle",
        legend=False,
        linewidth=2,
        ci=None,
        alpha=0.5,
        palette="mako_r",
    )

    plot_fl = plt.twinx()
    sns.lineplot(
        data=spectra_fl[
            (spectra_fl.gene == str(gene)) & (spectra_fl.treatment == "urea")
        ],
        x="wavelength",
        y="signal_normalized",
        hue="cycle",
        legend=False,
        linewidth=2,
        ci=None,
        alpha=0.5,
        palette=sns.cubehelix_palette(start=0, rot=-0.5, as_cmap=True),
    )

    plt.title("cgreWT" if gene == "cgre" else f"cgre{gene}", fontsize=18)
    plot.set_xlabel("Wavelength [nm]", fontsize=14)
    plot.set_ylabel("Signal (absorbance)", fontsize=14)
    plot_fl.set_ylabel("Signal (fluorescence)", fontsize=14)
    i += 1

plt.tight_layout()
plt.savefig("fl_abs_timeline_urea.png", dpi=300)

# Changes of signal intensity over number of cycles

Quantifying Urea "melting temperature".

A couple of methods:
- taking first derivative of the plots and taking it's maximum - this does not work, as the denaturation starts right away, the maximum value always on the first cycle?
- taking the first derivative of the plots on their linear part
- taking the number of the cycles at which the fluorescence is halved

## Absorbance

### In PBS

In [ ]:
plt.figure(figsize=(20, 4), dpi=200)
sns.lineplot(
    data=spectra_abs[
        (spectra_abs.treatment == "pbs")
        & (spectra_abs.peak == True)
        & (spectra_abs.cycle <= 100)
    ],
    x="cycle",
    y="signal_normalized",
    ci="sd",
    hue="gene",
    hue_order=[
        str(x)
        for x in (
            "cgre",
            1338,
            132,
            9708,
            4111,
            2880,
            3224,
            575,
            900,
            83,
            626,
            121,
            985,
            13,
            911,
            567,
            1414,
        )
    ],
    palette=sns.hls_palette(17, s=0.5),
)
plt.legend(frameon=False, bbox_to_anchor=(1, 0.5), loc="center left")

### In Urea

In [ ]:
plt.figure(figsize=(20, 4), dpi=200)
sns.lineplot(
    data=spectra_abs[
        (spectra_abs.treatment == "pbs")
        & (spectra_abs.peak == True)
        & (spectra_abs.cycle <= 100)
    ],
    x="cycle",
    y="signal_normalized",
    ci="sd",
    hue="gene",
    hue_order=[
        str(x)
        for x in (
            "cgre",
            1338,
            132,
            9708,
            4111,
            2880,
            3224,
            575,
            900,
            83,
            626,
            121,
            985,
            13,
            911,
            567,
            1414,
        )
    ],
    palette=sns.hls_palette(17, s=0.5),
)
plt.legend(frameon=False, bbox_to_anchor=(1, 0.5), loc="center left")

In [ ]:
plt.figure(figsize=(9, 15), dpi=200)
i = 1
for gene in (
    "cgre",
    1338,
    132,
    9708,
    4111,
    2880,
    3224,
    575,
    900,
    83,
    626,
    121,
    985,
    13,
    911,
    567,
    1414,
):
    plot = plt.subplot(6, 3, i)

    sns.lineplot(
        data=spectra_abs[
            (spectra_abs.treatment == "urea")
            & (spectra_abs.peak == True)
            & (spectra_abs.cycle <= 100)
            & (spectra_abs.gene == str(gene))
        ],
        x="cycle",
        y="signal_normalized",
        ci="sd",
        color=sns.hls_palette(17, s=0.5)[i - 1],
    )

    plt.title(f"{gene}", fontsize=18)
    plot.set_xlabel("Cycle", fontsize=14)
    plot.set_ylabel("Signal (9M urea)", fontsize=14)
    plt.yticks(ticks=[0, 0.25, 0.50, 0.75, 1.])
    #plt.xticks(ticks=list(range(0, 101, 5)), rotation=45, fontsize=8)

    i += 1

plt.tight_layout()

In [ ]:
plt.figure(figsize=(9, 15), dpi=200)
i = 1
for gene in (
    "cgre",
    1338,
    132,
    9708,
    4111,
    2880,
    3224,
    575,
    900,
    83,
    626,
    121,
    985,
    13,
    911,
    567,
    1414,
):
    plot = plt.subplot(6, 3, i)

    sns.lineplot(
        data=spectra_abs[
            (spectra_abs.treatment == "urea")
            & (spectra_abs.peak == True)
            & (spectra_abs.cycle <= 10)
            & (spectra_abs.gene == str(gene))
        ],
        x="cycle",
        y="signal_normalized",
        ci="sd",
        color=sns.hls_palette(17, s=0.5)[i - 1],
    )

    plt.title(f"{gene}", fontsize=18)
    plt.xticks(ticks=list(range(0, 10, 1)))
    
    plot.set_xlabel("Cycle", fontsize=14)
    plot.set_ylabel("Signal (9M urea)", fontsize=14)
    plt.yticks(ticks=[0, 0.25, 0.50, 0.75, 1.])

    i += 1

plt.tight_layout()

**1st derivative** 

In [ ]:
# Taking the peaks of the urea-treated genes, first 100 cycles, and grouping over genes and cycles, taking mean of the replicates
abs_before_diff = (
    spectra_abs[
        (spectra_abs.treatment == "urea")
        & (spectra_abs.peak == True)
        & (spectra_abs.cycle <= 100)
    ]
    .groupby(["gene", "cycle"], as_index=False)
    .mean()
)
# Taking the 1st derivative of the signal
abs_diff = (
    pd.DataFrame(
        abs_before_diff.groupby("gene")["signal_normalized"].apply(lambda x: np.diff(x))
    )
    .explode("signal_normalized")
    .reset_index()
    .rename(columns={"signal_normalized": "df_signal"})
)
# Taking cycles as the cumulative count of the genes
abs_diff["cycle"] = abs_diff.groupby("gene").cumcount()

In [ ]:
plt.figure(figsize=[20, 4], dpi=200)
sns.lineplot(
    data=abs_diff,
    x="cycle",
    y="df_signal",
    ci="sd",
    hue="gene",
    hue_order=[
        str(x)
        for x in (
            "cgre",
            1338,
            132,
            9708,
            4111,
            2880,
            3224,
            575,
            900,
            83,
            626,
            121,
            985,
            13,
            911,
            567,
            1414,
        )
    ],
    palette=sns.hls_palette(17, s=0.5),
)
plt.ylim(0, 0.1)
plt.legend(frameon=False, bbox_to_anchor=(1, 0.5), loc="center left")

#### Max derivative
We see, the graphs are sad. Let's take the max of the derivative for each gene:

In [ ]:
abs_diff.groupby("gene").max()["df_signal"]

#### Slope

Let's take the slope over the linear part of the graph:

In [ ]:
abs_lin_part_dict = {
    "cgre": [1, 35],
    # Plate 0
    "9708": [1, 20],
    "4111": [1, 20],
    "132": [1, 20],
    "1338": [1, 25],
    # Plate 1
    "83": [1, 9],
    "2880": [1, 20],
    "3224": [0, 40],
    "121": [2, 20],
    # Plate 2
    "626": [1, 6],
    "1414": [1, 60],
    "911": [0, 3],
    "900": [1, 60],
    # Plate 3
    "575": [1, 25],
    "985": [1, 20],
    # Plate 4
    "13": [1, 40],
    "567": [0, 2],
}

In [ ]:
abs_slope_per_gene = {}
for key, values in abs_lin_part_dict.items():
    abs_df_per_gene = abs_before_diff[
        (abs_before_diff.gene == key)
        & (abs_before_diff.cycle >= values[0])
        & (abs_before_diff.cycle <= values[1])
    ]
    slope = linregress(abs_df_per_gene["cycle"], abs_df_per_gene["signal_normalized"])[
        0
    ]

    abs_slope_per_gene[key] = slope

In [ ]:
abs_slope_per_gene

**Idea: make a graph with the slope and the fluorescence curves for each gene, showcasing the correlation**

#### Cycle at 1/2

Here we take a look at the number of the cycle where the value of the fluorescence is halved:

In [ ]:
spectra_abs.head()

In [ ]:
abs_urea_peaks = (
    spectra_abs[(spectra_abs.treatment == "urea") & (spectra_abs.peak == True)]
    .groupby(["gene", "cycle"], as_index=False)
    .mean()
)

In [ ]:
abs_gene_half_2_cycle = (
    abs_urea_peaks[abs_urea_peaks.signal_normalized <= 0.5]
    .groupby("gene")
    .min()["cycle"]
    .to_dict()
)

In [ ]:
abs_gene_half_2_cycle

Genes that have never reached the half:

In [ ]:
abs_res_genes = set(abs_lin_part_dict.keys()) - set(abs_gene_half_2_cycle.keys())

In [ ]:
abs_urea_peaks[abs_urea_peaks.gene.isin(abs_res_genes)].groupby("gene").max()

121 and 2880 reach half - 107 +, 985 - 133 +

In [ ]:
abs_gene_half_2_cycle.update({"121": 107, "2880": 107, "985": 133})

In [ ]:
abs_res_genes = set(abs_lin_part_dict.keys()) - set(abs_gene_half_2_cycle.keys())

In [ ]:
abs_res_genes

### Fitting curves

In [ ]:
fig = plt.figure(figsize=[9, 15], dpi=200)
i = 1
metrics_per_gene = {}
sigm_params = {}

for gene in (
    "cgre",
    1338,
    132,
    9708,
    4111,
    2880,
    3224,
    575,
    900,
    83,
    626,
    121,
    985,
    13,
    911,
    567,
    1414,
):
    plot = plt.subplot(6, 3, i)

    abs_urea_peak = spectra_abs[
        (spectra_abs.treatment == "urea")
        & (spectra_abs.peak == True)
        & (spectra_abs.cycle <= 100)
        & (spectra_abs.gene == str(gene))
    ]
    peak_wv = np.unique(abs_urea_peak["wavelength"])[0]

    sns.lineplot(
        data=abs_urea_peak,
        x="cycle",
        y="signal_normalized",
        ci="sd",
        label=f"Absorbance\nat {peak_wv} $nm$",
        color=sns.color_palette("mako", n_colors=4)[0],
    )

    mean_abs_urea = abs_urea_peak.groupby("cycle", as_index=False).mean()
    x = mean_abs_urea.cycle + 1
    y = mean_abs_urea.signal_normalized

    # Sigmoid
    # Initial params
    sigm_p0 = [max(y), np.median(x), 1, min(y)]
    # Fitting
    sigm_popt, sigm_pcov = curve_fit(sigmoid, x, y, sigm_p0, method="lm", maxfev=10000)

    sigm_x = np.linspace(1, 100, 1000)
    sigm_y = sigmoid(sigm_x, *sigm_popt)
    sigm_line = plt.plot(
        sigm_x,
        sigm_y,
        #                          label='$sigmoid$',
        c=sns.color_palette("mako", n_colors=4)[1],
    )

    # Exponential
    # Initial params
    exp_p0 = [np.max(y), -1]
    # Fitting
    exp_popt, exp_pcov = curve_fit(exponential, x, y, exp_p0, method="lm")
    exp_x = np.linspace(1, 100, 1000)
    exp_y = exponential(exp_x, *exp_popt)
    exp_line = plt.plot(
        exp_x,
        exp_y,
        #                         label='$exp$',
        c=sns.color_palette("mako", n_colors=4)[2],
    )

    # Logarithmic
    # Initial params
    log_p0 = [1, 1, 0]
    # Fitting
    log_popt, log_pcov = curve_fit(logarithmic, x, y, log_p0, method="lm", maxfev=10000)

    log_x = np.linspace(1, 100, 1000)
    log_y = logarithmic(log_x, *log_popt)
    log_line = plt.plot(
        log_x,
        log_y,
        #                         label='$log$',
        c=sns.color_palette("mako", n_colors=4)[3],
    )

    # Non-parametric correlation coeffecient (no presumptions about data)
    sigm_pred_y = sigmoid(x, *sigm_popt)
    exp_pred_y = exponential(x, *exp_popt)
    log_pred_y = logarithmic(x, *log_popt)
    metrics_per_gene[str(gene)] = {
        "kendall_sigm": kendalltau(y, sigm_pred_y).correlation,
        "kendall_exp": kendalltau(y, exp_pred_y).correlation,
        "kendall_log": kendalltau(y, log_pred_y).correlation,
        "mse_sigm": mean_squared_error(y, sigm_pred_y),
        "mse_exp": mean_squared_error(y, exp_pred_y),
        "mse_log": mean_squared_error(y, log_pred_y),
    }

    # Saving params for sigm, as the best fitting
    sigm_params[gene] = sigm_popt

    # Limits
    plt.xlabel(None)
    plt.ylabel(None)
    plt.xticks(ticks=list(range(0, 101, 20)))
    plt.ylim(-0.1, 1.2)
    #     plt.yticks(ticks=np.linspace(0, 1, 11))

    # Labelling
    plt.title(f"cgre{gene if gene != 'cgre' else ''}", fontsize=16)
    plt.legend(loc="upper right", frameon=False, fontsize=10, labelspacing=0.2)

    i += 1


fig.legend(
    handles=[sigm_line[0], exp_line[0], log_line[0]],
    labels=["$sigmoid$", "$exp$", "$log$"],
    bbox_to_anchor=(0.867, 0.175),
    borderaxespad=0,
    fontsize=14,
    frameon=False,
)
fig.supxlabel("Cycle", fontsize=18)
fig.supylabel("Normalized absorbance", fontsize=18)

plt.tight_layout()
plt.savefig(Path(".", "abs_curve_fitting.png"))

Making cycle into time:

In [ ]:
spectra_abs["time"] = spectra_abs["cycle"] * 40 / 60

In [ ]:
fig = plt.figure(figsize=[9, 15], dpi=200)
i = 1
metrics_per_gene = {}
sigm_params = {}

for gene in (
    "cgre",
    1338,
    132,
    9708,
    4111,
    2880,
    3224,
    575,
    900,
    83,
    626,
    121,
    985,
    13,
    911,
    567,
    1414,
):
    plot = plt.subplot(6, 3, i)

    abs_urea_peak = spectra_abs[
        (spectra_abs.treatment == "urea")
        & (spectra_abs.peak == True)
        & (spectra_abs.time <= 70)
        & (spectra_abs.gene == str(gene))
    ]
    peak_wv = np.unique(abs_urea_peak["wavelength"])[0]

    sns.lineplot(
        data=abs_urea_peak,
        x="time",
        y="signal_normalized",
        ci="sd",
        label=f"Absorbance\nat {peak_wv} $nm$",
        color=sns.color_palette("mako", n_colors=4)[0],
    )

    mean_abs_urea = abs_urea_peak.groupby("time", as_index=False).mean()
    x = mean_abs_urea.time + 1
    y = mean_abs_urea.signal_normalized

    # Sigmoid
    # Initial params
    sigm_p0 = [max(y), np.median(x), 1, min(y)]
    # Fitting
    sigm_popt, sigm_pcov = curve_fit(sigmoid, x, y, sigm_p0, method="lm", maxfev=10000)

    sigm_x = np.linspace(1, 70, 1000)
    sigm_y = sigmoid(sigm_x, *sigm_popt)
    sigm_line = plt.plot(
        sigm_x,
        sigm_y,
        #                          label='$sigmoid$',
        c=sns.color_palette("mako", n_colors=4)[1],
    )

    # Exponential
    # Initial params
    exp_p0 = [np.max(y), -1]
    # Fitting
    exp_popt, exp_pcov = curve_fit(exponential, x, y, exp_p0, method="lm")
    exp_x = np.linspace(1, 70, 1000)
    exp_y = exponential(exp_x, *exp_popt)
    exp_line = plt.plot(
        exp_x,
        exp_y,
        #                         label='$exp$',
        c=sns.color_palette("mako", n_colors=4)[2],
    )

    # Logarithmic
    # Initial params
    log_p0 = [1, 1, 0]
    # Fitting
    log_popt, log_pcov = curve_fit(logarithmic, x, y, log_p0, method="lm", maxfev=10000)

    log_x = np.linspace(1, 70, 1000)
    log_y = logarithmic(log_x, *log_popt)
    log_line = plt.plot(
        log_x,
        log_y,
        #                         label='$log$',
        c=sns.color_palette("mako", n_colors=4)[3],
    )

    # Non-parametric correlation coeffecient (no presumptions about data)
    sigm_pred_y = sigmoid(x, *sigm_popt)
    exp_pred_y = exponential(x, *exp_popt)
    log_pred_y = logarithmic(x, *log_popt)
    metrics_per_gene[str(gene)] = {
        "kendall_sigm": kendalltau(y, sigm_pred_y).correlation,
        "kendall_exp": kendalltau(y, exp_pred_y).correlation,
        "kendall_log": kendalltau(y, log_pred_y).correlation,
        "mse_sigm": mean_squared_error(y, sigm_pred_y),
        "mse_exp": mean_squared_error(y, exp_pred_y),
        "mse_log": mean_squared_error(y, log_pred_y),
    }

    # Saving params for sigm, as the best fitting
    sigm_params[gene] = sigm_popt

    # Limits
    plt.xlabel(None)
    plt.ylabel(None)
    plt.xticks(ticks=list(range(0, 71, 10)))
    plt.ylim(-0.1, 1.2)
    #     plt.yticks(ticks=np.linspace(0, 1, 11))

    # Labelling
    plt.title(f"cgre{gene if gene != 'cgre' else 'WT'}", fontsize=16)
    plt.legend(loc="upper right", frameon=False, fontsize=10, labelspacing=0.2)

    i += 1


fig.legend(
    handles=[sigm_line[0], exp_line[0], log_line[0]],
    labels=["$sigmoid$", "$exp$", "$log$"],
    bbox_to_anchor=(0.867, 0.175),
    borderaxespad=0,
    fontsize=14,
    frameon=False,
)
fig.supxlabel("Time [h]", fontsize=18)
fig.supylabel("Normalized absorbance", fontsize=18)

plt.tight_layout()
plt.savefig(Path(".", "abs_curve_fitting_time.png"))

In [ ]:
pd.DataFrame(metrics_per_gene).T

## Sigmoid best fitting -> Inverse

Inverse of the sigmoid function:
$$
    y = \frac{L}{  (1 + e^{-k \cdot (x-x_0)})} + b
$$

$$
(y - b) = \frac{L}{  (1 + e^{-k \cdot (x-x_0)})} \\
 (1 + e^{-k \cdot (x-x_0)}) = \frac{L}{ (y - b) } \\
  e^{-k \cdot (x-x_0)} = \frac{L}{ (y - b) } - 1 \\
  -k \cdot (x-x_0) = ln(\frac{L}{ (y - b) } - 1) \\
  x = - \frac{ln(\frac{L}{ (y - b) } - 1)}{k} + x_0 \\
$$

In [ ]:
# def inverse_sigmoid(y, L ,x0, k, b):
#     x = -np.log(L/(y-b) - 1) / k + x0
#     return x

In [ ]:
half_ur_abs = {}
for gene, sigm_popt in sigm_params.items():
    abs_urea_peak = (
        spectra_abs[
            (spectra_abs.treatment == "urea")
            & (spectra_abs.peak == True)
            & (spectra_abs.time <= 70)
            & (spectra_abs.gene == str(gene))
        ]
        .groupby("time", as_index=False)
        .mean()
    )
    half_ur_abs[str(gene)] = inverse_sigmoid(0.5, *sigm_popt)

In [ ]:
half_ur_abs

In [ ]:
del half_ur_abs["cgre"]

## Fluorescence

### In PBS

In [ ]:
plt.figure(figsize=(20, 4), dpi=200)
sns.lineplot(
    data=spectra_fl[
        (spectra_fl.treatment == "pbs")
        & (spectra_fl.peak == True)
        & (spectra_fl.cycle <= 130)
    ],
    x="cycle",
    y="signal_normalized",
    ci="sd",
    hue="gene",
    hue_order=[
        str(x)
        for x in (
            "cgre",
            1338,
            132,
            9708,
            4111,
            2880,
            3224,
            575,
            900,
            83,
            626,
            121,
            985,
            13,
            911,
            567,
            1414,
        )
    ],
    palette=sns.hls_palette(17, s=0.5),
)
plt.legend(frameon=False, bbox_to_anchor=(1, 0.5), loc="center left")

### In Urea

In [ ]:
plt.figure(figsize=(20, 4), dpi=200)
sns.lineplot(
    data=spectra_fl[
        (spectra_fl.treatment == "urea")
        & (spectra_fl.peak == True)
        & (spectra_fl.cycle <= 130)
    ],
    x="cycle",
    y="signal_normalized",
    ci="sd",
    hue="gene",
    hue_order=[
        str(x)
        for x in (
            "cgre",
            1338,
            132,
            9708,
            4111,
            2880,
            3224,
            575,
            900,
            83,
            626,
            121,
            985,
            13,
            911,
            567,
            1414,
        )
    ],
    palette=sns.hls_palette(17, s=0.5),
)
plt.legend(frameon=False, bbox_to_anchor=(1, 0.5), loc="center left")

In [ ]:
plt.figure(figsize=(30, 9), dpi=200)
i = 1
for gene in (
    "cgre",
    1338,
    132,
    9708,
    4111,
    2880,
    3224,
    575,
    900,
    83,
    626,
    121,
    985,
    13,
    911,
    567,
    1414,
):
    plot = plt.subplot(3, 6, i)

    sns.lineplot(
        data=spectra_fl[
            (spectra_fl.treatment == "urea")
            & (spectra_fl.peak == True)
            & (spectra_fl.cycle <= 130)
            & (spectra_fl.gene == str(gene))
        ],
        x="cycle",
        y="signal_normalized",
        ci="sd",
        color=sns.hls_palette(17, s=0.5)[i - 1],
    )

    plt.title(f"{gene}", fontsize=18)
    plt.xticks(ticks=list(range(0, 131, 5)))

    i += 1

plt.tight_layout()

In [ ]:
plt.figure(figsize=(30, 9), dpi=200)
i = 1
for gene in (
    "cgre",
    1338,
    132,
    9708,
    4111,
    2880,
    3224,
    575,
    900,
    83,
    626,
    121,
    985,
    13,
    911,
    567,
    1414,
):
    plot = plt.subplot(3, 6, i)

    sns.lineplot(
        data=spectra_fl[
            (spectra_fl.treatment == "urea")
            & (spectra_fl.peak == True)
            & (spectra_fl.cycle <= 10)
            & (spectra_fl.gene == str(gene))
        ],
        x="cycle",
        y="signal_normalized",
        ci="sd",
        color=sns.hls_palette(17, s=0.5)[i - 1],
    )

    plt.title(f"{gene}", fontsize=18)
    plt.xticks(ticks=list(range(0, 10, 1)))

    i += 1

plt.tight_layout()

**1st derivative** 

In [ ]:
# Taking the peaks of the urea-treated genes, first 130 cycles, and grouping over genes and cycles, taking mean of the replicates
fl_before_diff = (
    spectra_fl[
        (spectra_fl.treatment == "urea")
        & (spectra_fl.peak == True)
        & (spectra_fl.cycle <= 130)
    ]
    .groupby(["gene", "cycle"], as_index=False)
    .mean()
)
# Taking the 1st derivative of the signal
fl_diff = (
    pd.DataFrame(
        fl_before_diff.groupby("gene")["signal_normalized"].apply(lambda x: -np.diff(x))
    )
    .explode("signal_normalized")
    .reset_index()
    .rename(columns={"signal_normalized": "df_signal"})
)
# Taking cycles as the cumulative count of the genes
fl_diff["cycle"] = fl_diff.groupby("gene").cumcount()

In [ ]:
plt.figure(figsize=(20, 4), dpi=200)
sns.lineplot(
    data=fl_diff,
    x="cycle",
    y="df_signal",
    ci="sd",
    hue="gene",
    hue_order=[
        str(x)
        for x in (
            "cgre",
            1338,
            132,
            9708,
            4111,
            2880,
            3224,
            575,
            900,
            83,
            626,
            121,
            985,
            13,
            911,
            567,
            1414,
        )
    ],
    palette=sns.hls_palette(17, s=0.5),
)
plt.legend(frameon=False, bbox_to_anchor=(1, 0.5), loc="center left")

#### Max derivative
We see, the graphs are sad. Let's take the max of the derivative for each gene:

In [ ]:
fl_diff.groupby("gene").max()["df_signal"]

#### Slope

Let's take the slope over the linear part of the graph:

In [ ]:
fl_lin_part_dict = {
    "cgre": [1, 10],
    # Plate 0
    "9708": [2, 25],
    "4111": [1, 35],
    "132": [1, 20],
    "1338": [1, 7],
    # Plate 1
    "83": [0, 4],
    "2880": [1, 30],
    "3224": [0, 15],
    "121": [0, 5],
    # Plate 2
    "626": [0, 3],
    "1414": [1, 50],
    "911": [0, 2],
    "900": [0, 5],  # ?
    # Plate 3
    "575": [0, 10],
    "985": [1, 120],
    # Plate 4
    "13": [1, 100],
    "567": [0, 1],
}

In [ ]:
fl_slope_per_gene = {}
for key, values in fl_lin_part_dict.items():
    fl_df_per_gene = fl_before_diff[
        (fl_before_diff.gene == key)
        & (fl_before_diff.cycle >= values[0])
        & (fl_before_diff.cycle <= values[1])
    ]
    slope = linregress(fl_df_per_gene["cycle"], fl_df_per_gene["signal_normalized"])[0]

    fl_slope_per_gene[key] = slope

In [ ]:
fl_slope_per_gene

**Idea: make a graph with the slope and the fluorescence curves for each gene, showcasing the correlation**

#### Cycle at 1/2

Here we take a look at the number of the cycle where the value of the fluorescence is halved:

In [ ]:
spectra_fl.head()

In [ ]:
fl_urea_peaks = (
    spectra_fl[(spectra_fl.treatment == "urea") & (spectra_fl.peak == True)]
    .groupby(["gene", "cycle"], as_index=False)
    .mean()
)

In [ ]:
fl_gene_half_2_cycle = (
    fl_urea_peaks[fl_urea_peaks.signal_normalized <= 0.5]
    .groupby("gene")
    .min()["cycle"]
    .to_dict()
)

In [ ]:
fl_gene_half_2_cycle

Genes that have never reached the half:

In [ ]:
fl_res_genes = set(fl_lin_part_dict.keys()) - set(fl_gene_half_2_cycle.keys())

In [ ]:
fl_res_genes

In [ ]:
fl_urea_peaks[fl_urea_peaks.gene.isin(fl_res_genes)].groupby("gene").max()

All genes reach half! 

### Fitting curves

In [ ]:
fig = plt.figure(figsize=[9, 15], dpi=200)
i = 1
metrics_per_gene = {}
sigm_params = {}

for gene in (
    "cgre",
    1338,
    132,
    9708,
    4111,
    2880,
    3224,
    575,
    900,
    83,
    626,
    121,
    985,
    13,
    911,
    567,
    1414,
):
    plot = plt.subplot(6, 3, i)

    fl_urea_peak = spectra_fl[
        (spectra_fl.treatment == "urea")
        & (spectra_fl.peak == True)
        & (spectra_fl.cycle <= 135)
        & (spectra_fl.gene == str(gene))
    ]
    peak_wv = np.unique(fl_urea_peak["wavelength"])[0]

    sns.lineplot(
        data=fl_urea_peak,
        x="cycle",
        y="signal_normalized",
        ci="sd",
        label=f"Fluroscence\nat {peak_wv} $nm$",
        color=sns.color_palette("mako", n_colors=4)[0],
    )

    mean_abs_urea = fl_urea_peak.groupby("cycle", as_index=False).mean()
    x = mean_abs_urea.cycle + 1
    y = mean_abs_urea.signal_normalized

    # Sigmoid
    # Initial params
    sigm_p0 = [max(y), np.median(x), 1, min(y)]
    # Fitting
    sigm_popt, sigm_pcov = curve_fit(sigmoid, x, y, sigm_p0, method="lm", maxfev=10000)

    sigm_x = np.linspace(1, 135, 1000)
    sigm_y = sigmoid(sigm_x, *sigm_popt)
    sigm_line = plt.plot(
        sigm_x,
        sigm_y,
        #                          label='$sigmoid$',
        c=sns.color_palette("mako", n_colors=4)[1],
    )

    # Exponential
    # Initial params
    exp_p0 = [np.max(y), -1]
    # Fitting
    exp_popt, exp_pcov = curve_fit(exponential, x, y, exp_p0, method="lm")
    exp_x = np.linspace(1, 135, 1000)
    exp_y = exponential(exp_x, *exp_popt)
    exp_line = plt.plot(
        exp_x,
        exp_y,
        #                         label='$exp$',
        c=sns.color_palette("mako", n_colors=4)[2],
    )

    # Logarithmic
    # Initial params
    log_p0 = [1, 1, 0]
    # Fitting
    log_popt, log_pcov = curve_fit(
        logarithmic, x, y, log_p0, method="lm", maxfev=100000
    )

    log_x = np.linspace(1, 135, 1000)
    log_y = logarithmic(log_x, *log_popt)
    log_line = plt.plot(
        log_x,
        log_y,
        #                         label='$log$',
        c=sns.color_palette("mako", n_colors=4)[3],
    )

    # Non-parametric correlation coeffecient (no presumptions about data)
    sigm_pred_y = sigmoid(x, *sigm_popt)
    exp_pred_y = exponential(x, *exp_popt)
    log_pred_y = logarithmic(x, *log_popt)
    metrics_per_gene[str(gene)] = {
        "kendall_sigm": kendalltau(y, sigm_pred_y).correlation,
        "kendall_exp": kendalltau(y, exp_pred_y).correlation,
        "kendall_log": kendalltau(y, log_pred_y).correlation,
        "mse_sigm": mean_squared_error(y, sigm_pred_y),
        "mse_exp": mean_squared_error(y, exp_pred_y),
        "mse_log": mean_squared_error(y, log_pred_y),
    }

    # Saving params for sigm, as the best fitting
    sigm_params[gene] = sigm_popt

    # Limits
    plt.xlabel(None)
    plt.ylabel(None)
    plt.xticks(ticks=list(range(0, 141, 20)))
    plt.ylim(-0.1, 1.2)
    #     plt.yticks(ticks=np.linspace(0, 1, 11))

    # Labelling
    plt.title(f"cgre{gene if gene != 'cgre' else ''}", fontsize=16)
    plt.legend(loc="upper right", frameon=False, fontsize=10, labelspacing=0.2)

    i += 1


fig.legend(
    handles=[sigm_line[0], exp_line[0], log_line[0]],
    labels=["$sigmoid$", "$exp$", "$log$"],
    bbox_to_anchor=(0.867, 0.175),
    borderaxespad=0,
    fontsize=14,
    frameon=False,
)
fig.supxlabel("Cycle", fontsize=18)
fig.supylabel("Normalized fluorescence", fontsize=18)

plt.tight_layout()
plt.savefig(Path(".", "fl_curve_fitting.png"))

Making cycle into time:

In [ ]:
spectra_fl["time"] = spectra_fl["cycle"] * 25 / 60

In [ ]:
fig = plt.figure(figsize=[9, 15], dpi=200)
i = 1
metrics_per_gene = {}
sigm_params = {}

for gene in (
    "cgre",
    1338,
    132,
    9708,
    4111,
    2880,
    3224,
    575,
    900,
    83,
    626,
    121,
    985,
    13,
    911,
    567,
    1414,
):
    plot = plt.subplot(6, 3, i)

    fl_urea_peak = spectra_fl[
        (spectra_fl.treatment == "urea")
        & (spectra_fl.peak == True)
        & (spectra_fl.time <= 60)
        & (spectra_fl.gene == str(gene))
    ]
    peak_wv = np.unique(fl_urea_peak["wavelength"])[0]

    sns.lineplot(
        data=fl_urea_peak,
        x="time",
        y="signal_normalized",
        ci="sd",
        label=f"Fluorescence\nat {peak_wv} $nm$",
        color=sns.color_palette("mako", n_colors=4)[0],
    )

    mean_abs_urea = fl_urea_peak.groupby("time", as_index=False).mean()
    x = mean_abs_urea.time + 1
    y = mean_abs_urea.signal_normalized

    # Sigmoid
    # Initial params
    sigm_p0 = [max(y), np.median(x), 1, min(y)]
    # Fitting
    sigm_popt, sigm_pcov = curve_fit(sigmoid, x, y, sigm_p0, method="lm", maxfev=10000)

    sigm_x = np.linspace(1, 60, 1000)
    sigm_y = sigmoid(sigm_x, *sigm_popt)
    sigm_line = plt.plot(
        sigm_x,
        sigm_y,
        #                          label='$sigmoid$',
        c=sns.color_palette("mako", n_colors=4)[1],
    )

    # Exponential
    # Initial params
    exp_p0 = [np.max(y), -1]
    # Fitting
    exp_popt, exp_pcov = curve_fit(exponential, x, y, exp_p0, method="lm")
    exp_x = np.linspace(1, 60, 1000)
    exp_y = exponential(exp_x, *exp_popt)
    exp_line = plt.plot(
        exp_x,
        exp_y,
        #                         label='$exp$',
        c=sns.color_palette("mako", n_colors=4)[2],
    )

    # Logarithmic
    # Initial params
    log_p0 = [1, 1, 0]
    # Fitting
    log_popt, log_pcov = curve_fit(logarithmic, x, y, log_p0, method="lm", maxfev=10000)

    log_x = np.linspace(1, 60, 1000)
    log_y = logarithmic(log_x, *log_popt)
    log_line = plt.plot(
        log_x,
        log_y,
        #                         label='$log$',
        c=sns.color_palette("mako", n_colors=4)[3],
    )

    # Non-parametric correlation coeffecient (no presumptions about data)
    sigm_pred_y = sigmoid(x, *sigm_popt)
    exp_pred_y = exponential(x, *exp_popt)
    log_pred_y = logarithmic(x, *log_popt)
    metrics_per_gene[gene] = {
        "kendall_sigm": kendalltau(y, sigm_pred_y).correlation,
        "kendall_exp": kendalltau(y, exp_pred_y).correlation,
        "kendall_log": kendalltau(y, log_pred_y).correlation,
        "mse_sigm": mean_squared_error(y, sigm_pred_y),
        "mse_exp": mean_squared_error(y, exp_pred_y),
        "mse_log": mean_squared_error(y, log_pred_y),
    }

    # Saving params for sigm, as the best fitting
    sigm_params[gene] = sigm_popt

    # Limits
    plt.xlabel(None)
    plt.ylabel(None)
    plt.xticks(ticks=list(range(0, 61, 15)))
    plt.ylim(-0.1, 1.2)

    # Labelling
    plt.title(f"cgre{gene if gene != 'cgre' else ''}", fontsize=16)
    plt.legend(loc="upper right", frameon=False, fontsize=10, labelspacing=0.2)

    i += 1


fig.legend(
    handles=[sigm_line[0], exp_line[0], log_line[0]],
    labels=["$sigmoid$", "$exp$", "$log$"],
    bbox_to_anchor=(0.867, 0.175),
    borderaxespad=0,
    fontsize=14,
    frameon=False,
)
fig.supxlabel("Time [h]", fontsize=18)
fig.supylabel("Normalized fluorescence", fontsize=18)

plt.tight_layout()
plt.savefig(Path(".", "fl_curve_fitting_time.png"))

In [ ]:
pd.DataFrame(metrics_per_gene).T

## Sigmoid best fitting -> Inverse

Inverse of the sigmoid function:
$$
    y = \frac{L}{  (1 + e^{-k \cdot (x-x_0)})} + b
$$

$$
(y - b) = \frac{L}{  (1 + e^{-k \cdot (x-x_0)})} \\
 (1 + e^{-k \cdot (x-x_0)}) = \frac{L}{ (y - b) } \\
  e^{-k \cdot (x-x_0)} = \frac{L}{ (y - b) } - 1 \\
  -k \cdot (x-x_0) = ln(\frac{L}{ (y - b) } - 1) \\
  x = - \frac{ln(\frac{L}{ (y - b) } - 1)}{k} + x_0 \\
$$

In [ ]:
half_ur_fl = {}
for gene, sigm_popt in sigm_params.items():
    fl_urea_peak = (
        spectra_fl[
            (spectra_fl.treatment == "urea")
            & (spectra_fl.peak == True)
            & (spectra_abs.time <= 60)
            & (spectra_fl.gene == str(gene))
        ]
        .groupby("time", as_index=False)
        .mean()
    )
    half_ur_fl[str(gene)] = inverse_sigmoid(0.5, *sigm_popt)

In [ ]:
half_ur_fl

In [ ]:
del half_ur_fl["cgre"]

## Metrics

### Correlation between fluorescence and absorbance for the same methods

#### Slope:

In [ ]:
df_slopes = pd.DataFrame(
    {"abs": pd.Series(abs_slope_per_gene), "fl": pd.Series(fl_slope_per_gene)}
)

In [ ]:
df_slopes.corr()

#### 1/2:

In [ ]:
df_halves = pd.DataFrame(
    {"abs": pd.Series(abs_gene_half_2_cycle), "fl": pd.Series(fl_gene_half_2_cycle)}
)

In [ ]:
df_halves.corr()

#### Curve fitting:

In [ ]:
df_curves = pd.DataFrame({"abs": pd.Series(half_ur_abs), "fl": pd.Series(half_ur_fl)})

In [ ]:
df_curves.corr()

### Correlationn between metrics

#### Slope + 1/2

In [ ]:
df_slopes["abs"].corr(df_halves["abs"])

In [ ]:
df_slopes["fl"].corr(df_halves["fl"])

#### Slope + curve fitting

In [ ]:
df_slopes["abs"].corr(df_curves["abs"])

In [ ]:
df_slopes["fl"].corr(df_curves["fl"])

#### 1/2 + curve fitting

In [ ]:
df_halves["abs"].corr(df_curves["abs"])

In [ ]:
df_halves["fl"].corr(df_curves["fl"])

## Correlation with the mutational robustness

In [ ]:
half_mut_rob_per_gene = {'cgreGFP': 2.3598224708239237,
 2880: 2.0475472322138573,
 575: 1.648929743078233,
 83: 0.4876710628582579,
 121: 2.087983470121052,
 13: 1.7655853131041812,
 567: 0.38997176295167435,
 3224: 1.364926200088591,
 900: 1.6385128907623048,
 626: 1.4664978833021196,
 985: 1.8639110762878979,
 911: 2.1317630470636817,
 1414: 4.7128750665062,
 1338: 0.9603722537683815,
 132: 3.2927772016988577,
 9708: 3.077040183325697,
 4111: 1.072414674710171}
half_mut_rob_per_gene = {
    str(key): value for key, value in half_mut_rob_per_gene.items()
}
df_half_mut_rob = pd.DataFrame(
    half_mut_rob_per_gene, index=["half_n_aamut"]
).T.sort_index()

In [ ]:
df_slopes_without_wt = df_slopes.loc[df_slopes.index != "cgre"].sort_index()
df_halves_without_wt = df_halves.loc[df_halves.index != "cgre"].sort_index()
df_curves_without_wt = df_curves.loc[df_curves.index != "cgre"].sort_index()

In [ ]:
from scipy.stats import pearsonr

### Slopes

In [ ]:
df_slopes_without_wt["abs"].corr(df_half_mut_rob["half_n_aamut"])

In [ ]:
common_ids = np.intersect1d(df_slopes_without_wt["abs"].index, df_half_mut_rob["half_n_aamut"].index)
pearsonr(df_slopes_without_wt["abs"][common_ids], df_half_mut_rob["half_n_aamut"][common_ids])

In [ ]:
df_slopes_without_wt["fl"].corr(df_half_mut_rob["half_n_aamut"])

In [ ]:
common_ids = np.intersect1d(df_slopes_without_wt["fl"].index, df_half_mut_rob["half_n_aamut"].index)
pearsonr(df_slopes_without_wt["fl"][common_ids], df_half_mut_rob["half_n_aamut"][common_ids])

### 1/2

In [ ]:
df_halves_without_wt["abs"].corr(df_half_mut_rob["half_n_aamut"])

In [ ]:
common_ids = np.intersect1d(df_halves_without_wt["abs"].index, df_half_mut_rob["half_n_aamut"].index)
pearsonr(df_halves_without_wt["abs"][common_ids], df_half_mut_rob["half_n_aamut"][common_ids])

In [ ]:
df_halves_without_wt["fl"].corr(df_half_mut_rob["half_n_aamut"])

In [ ]:
common_ids = np.intersect1d(df_halves_without_wt["fl"].index, df_half_mut_rob["half_n_aamut"].index)
pearsonr(df_halves_without_wt["fl"][common_ids], df_half_mut_rob["half_n_aamut"][common_ids])

### Curve fitting

In [ ]:
df_curves_without_wt["abs"].corr(df_half_mut_rob["half_n_aamut"])

In [ ]:
common_ids = np.intersect1d(df_curves_without_wt["abs"].index, df_half_mut_rob["half_n_aamut"].index)
pearsonr(df_curves_without_wt["abs"][common_ids], df_half_mut_rob["half_n_aamut"][common_ids])

In [ ]:
df_curves_without_wt["fl"].corr(df_half_mut_rob["half_n_aamut"])

In [ ]:
common_ids = np.intersect1d(df_curves_without_wt["fl"].index, df_half_mut_rob["half_n_aamut"].index)
pearsonr(df_curves_without_wt["fl"][common_ids], df_half_mut_rob["half_n_aamut"][common_ids])

In [ ]:
corr_results = {
                "Abs r": [0.37, 0.54, 0.37],
                "Abs p-value": [0.16, 0.03, 0.16],
                "Fl r": [0.45, 0.29, 0.29],
                "Fl p-value": [0.08,  0.28, 0.28]
                }

corr_results = pd.DataFrame(corr_results, index=["Slopes", "Halves", "Curves"])
corr_results.to_csv("corr_results.csv")

In [ ]:
corr_results